In [1]:
import threading
import time

def count():
    x = 0
    for _ in range(10**7):
        x += 1

# Run two threads
start = time.time()
t1 = threading.Thread(target=count)
t2 = threading.Thread(target=count)

t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()

print(f"Time taken with two threads: {end - start:.2f} seconds")


Time taken with two threads: 0.33 seconds


In [2]:
start = time.time()
count()
count()
end = time.time()
print(f"Time taken in single thread: {end - start:.2f} seconds")


Time taken in single thread: 0.38 seconds


In [3]:
import multiprocessing
import time

def count():
    x = 0
    for _ in range(10**7):
        x += 1

if __name__ == "__main__":
    start = time.time()
    p1 = multiprocessing.Process(target=count)
    p2 = multiprocessing.Process(target=count)

    p1.start()
    p2.start()
    p1.join()
    p2.join()
    end = time.time()

    print(f"Time taken with multiprocessing: {end - start:.2f} seconds")


Time taken with multiprocessing: 0.05 seconds


Traceback (most recent call last):
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "<string>", line 1, in <module>
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^AttributeError^^: ^Can't get attribute '

In [ ]:
import threading
import multiprocessing
import time

def count():
    x = 0
    for _ in range(10**7):
        x += 1

# 1️⃣ Single-threaded run
def single_thread_test():
    start = time.time()
    count()
    count()
    end = time.time()
    print(f"Single-threaded: {end - start:.2f} seconds")

# 2️⃣ Multi-threaded run (affected by GIL)
def multi_thread_test():
    start = time.time()
    t1 = threading.Thread(target=count)
    t2 = threading.Thread(target=count)
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    end = time.time()
    print(f"Multi-threaded (GIL): {end - start:.2f} seconds")

# 3️⃣ Multiprocessing run (runs in parallel)
def multi_process_test():
    start = time.time()
    p1 = multiprocessing.Process(target=count)
    p2 = multiprocessing.Process(target=count)
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    end = time.time()
    print(f"Multiprocessing: {end - start:.2f} seconds")


if __name__ == "__main__":
    print("Running benchmark tests...\n")
    single_thread_test()
    multi_thread_test()
    multi_process_test()


In [5]:
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

def count():
    x = 0
    for _ in range(10**7):
        x += 1
    return x

def run_threadpool():
    start = time.time()
    with ThreadPoolExecutor(max_workers=2) as executor:
        futures = [executor.submit(count) for _ in range(2)]
        for future in futures:
            future.result()
    end = time.time()
    print(f"ThreadPoolExecutor (GIL): {end - start:.2f} seconds")

def run_processpool():
    start = time.time()
    with ProcessPoolExecutor(max_workers=2) as executor:
        futures = [executor.submit(count) for _ in range(2)]
        for future in futures:
            future.result()
    end = time.time()
    print(f"ProcessPoolExecutor (parallel): {end - start:.2f} seconds")

if __name__ == "__main__":
    run_threadpool()
    run_processpool()


ThreadPoolExecutor (GIL): 0.31 seconds


Process SpawnProcess-5:
Process SpawnProcess-6:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/concurrent/futures/process.py", line 252, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'count' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/py

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [6]:
import time
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

def count_with_progress(_):
    x = 0
    for _ in tqdm(range(10**7), desc="Counting", leave=False):
        x += 1
    return x

def run_with_progress():
    start = time.time()
    with ProcessPoolExecutor(max_workers=2) as executor:
        futures = [executor.submit(count_with_progress, i) for i in range(2)]
        for future in tqdm(futures, desc="Overall Progress"):
            future.result()
    end = time.time()
    print(f"ProcessPoolExecutor with progress bars: {end - start:.2f} seconds")

if __name__ == "__main__":
    run_with_progress()


Overall Progress:   0%|          | 0/2 [00:00<?, ?it/s]Process SpawnProcess-7:
Process SpawnProcess-8:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/concurrent/futures/process.py", line 252, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zeynmehezmacbook/miniconda3/envs/geoenv/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'count_with_progress' on <module '__main__' (<class '_frozen_importlib.BuiltinImpor

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.